In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import matplotlib.ticker as mticker
import time
import csv

In [17]:
# https://www.kaggle.com/datasets/marwa80/userbehavior?resource=download
data_fn = '../data/taobao/UserBehavior.csv'
headers = ['user_id', 'item_id',  'category_id', 'behavior_type', 'timestamp']
df = pd.read_csv(data_fn, header=None, names=headers)

In [18]:
display(df)

,user_id,item_id,category_id,behavior_type,timestamp
0,1,2268318,2520377,pv,1511544070
1,1,2333346,2520771,pv,1511561733
2,1,2576651,149192,pv,1511572885
3,1,3830808,4181361,pv,1511593493
4,1,4365585,2520377,pv,1511596146
...,...,...,...,...,...
100150802,999999,4797808,11120,pv,1512293403
100150803,999999,4613472,4602841,pv,1512293766
100150804,999999,3647364,2304296,pv,1512293792
100150805,999999,1903801,2304296,pv,1512293827


In [25]:
def timestamp_to_timeseries(timestamps, start_time, end_time, bin_size):
    hist, _ = np.histogram(timestamps, bins=np.arange(start_time, end_time+bin_size, bin_size))
    return hist

def groupby(df, group_header='category_id', timestamp_header='timestamp'):
    np_df = df[[group_header, timestamp_header]].to_numpy()
    groups, groups_index = np.unique(np_df[:, 0], return_index=True)
    return np.split(np_df[:,1], groups_index[1:]), groups

def timeseries_to_csv(timeseries, group, csv_fn):
    timeseries_group = [group]
    timeseries_group.extend(list(timeseries))
    with open(csv_fn, 'a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(timeseries_group)

In [28]:
start_time = time.mktime(datetime.datetime(2017, 11, 25, 00, 00).timetuple())
end_time = time.mktime(datetime.datetime(2017, 12, 4, 0, 0).timetuple())
bin_size = 60*60 # in seconds
csv_fn = '../data/taobao/hourly_categoryid_timeseries.csv'
timestamp_header = 'timestamp'
group_header = 'category_id'

timeseries_to_csv(np.arange(start_time, end_time+bin_size, bin_size), group_header, csv_fn)
timestamps_grouped, groups= groupby(df, group_header, timestamp_header)
for timestamps, group in zip(timestamps_grouped, groups):
    timeseries = timestamp_to_timeseries(timestamps, start_time, end_time, bin_size)
    timeseries_to_csv(timeseries, group, csv_fn)